# Machine Learning Model Inference

This notebook allows you to perform inference using pre-trained machine learning models for Forecasting, Regression, and Classification tasks.

## 1. Import Libraries

First, let's import the necessary libraries. You may need to install some of these if you don't have them already. You can do this by running `pip install <library_name>` in your terminal.

In [1]:
import ipywidgets as widgets
from IPython.display import display

import os
import ray
import math
import dill
import pickle
import psutil
import datetime
import numpy as np
import pandas as pd

import xgboost as xgb
import tensorflow as tf
from neuralprophet import NeuralProphet
from statsmodels.tsa.seasonal import STL
from xgboost import XGBRegressor, XGBClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC, OneClassSVM, SVR
from statsmodels.tsa.arima.model import ARIMA, ARIMAResultsWrapper
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score, r2_score
from catboost import CatBoostRegressor, CatBoostClassifier, Pool, EFstrType
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, MinMaxScaler
from statsmodels.tsa.statespace.sarimax import SARIMAX, SARIMAXResultsWrapper

import warnings
warnings.filterwarnings('ignore')

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


## 2. User Inputs

Please provide the necessary inputs below.

In [2]:
style = {'description_width': 'initial'}

ml_task = widgets.Dropdown(
    options=['Regression', 'Classification', 'Forecasting'],
    value='Regression',
    description='Select ML Task:',
    disabled=False,
    style=style
)

model_path = widgets.FileUpload(
    accept='.pkl,.dill',
    multiple=False,
    description='Model Checkpoint'
)

input_data_path = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Inference CSV Data'
)

forecast_periods = widgets.IntText(
    value=0,
    description='Forecast Next N Periods:',
    disabled=False,
    style=style
)

start_date = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='Start Date:',
    disabled=False,
    style=style
)

end_date = widgets.Text(
    value='',
    placeholder='YYYY-MM-DD',
    description='End Date:',
    disabled=False,
    style=style
)

or_separator = widgets.HTML(
    value="""
    <div style='display: flex; align-items: center; margin: 10px 0;'>
        <hr style='flex-grow: 1; border-top: 1px solid #bbb;' />
        <span style='padding: 0 15px; font-weight: bold; color: #555;'>OR</span>
        <hr style='flex-grow: 1; border-top: 1px solid #bbb;' />
    </div>
    """,
    layout=widgets.Layout(width='100%')
)

forecasting_options = widgets.VBox([
    forecast_periods,
    or_separator,
    widgets.HBox([start_date, end_date])
])

def on_ml_task_change(change):
    if change['new'] == 'Forecasting':
        input_data_path.layout.display = 'none'
        forecasting_options.layout.display = 'flex'
    else:
        input_data_path.layout.display = 'flex'
        forecasting_options.layout.display = 'none'

ml_task.observe(on_ml_task_change, names='value')

display(ml_task, model_path, input_data_path, forecasting_options)

on_ml_task_change({'new': ml_task.value})

Dropdown(description='Select ML Task:', options=('Regression', 'Classification', 'Forecasting'), style=Descrip…

FileUpload(value=(), accept='.pkl,.dill', description='Model Checkpoint')

FileUpload(value=(), accept='.csv', description='Inference CSV Data')

## 3. Perform Inference

Run the cell below to perform inference based on your selections.

In [3]:
def perform_inference():
    if not model_path.value:
        print("Please upload a model checkpoint.")
        return

    model_file_info = list(model_path.value.values())[0]
    model_content = model_file_info['content']
    
    try:
        try:
            model_data = pickle.loads(model_content)
        except (pickle.UnpicklingError, dill.UnpicklingError):
            model_data = dill.loads(model_content)
    except Exception as e:
        print(f"Error loading model: {e}")
        return

    task = ml_task.value

    if task == 'Forecasting':
        model = model_data['model']
        test_end_date = model_data['testEndDate']
        frequency = model_data['timeIntervalFrequency']

        if start_date.value and end_date.value:
            forecast_range = pd.date_range(start=start_date.value, end=end_date.value, freq=frequency)
        else:
            forecast_range = pd.date_range(start=test_end_date, periods=forecast_periods.value + 1, freq=frequency)[1:]

        if 'neuralprophet' in str(type(model)):
            future = pd.DataFrame({'ds': forecast_range})
            forecast = model.predict(future)
            predictions = forecast[['ds', 'yhat1']].set_index('ds')
        if isinstance(model, (ARIMAResultsWrapper, SARIMAXResultsWrapper)):
            predictions = model.forecast(steps=len(forecast_range))
            predictions.index = forecast_range
        else:
            print("Unsupported forecasting model type.")
            return

        print("Forecasted Values:")
        display(predictions)

    elif task in ['Regression', 'Classification']:
        if not input_data_path.value:
            print("Please upload input CSV data.")
            return

        input_file_info = list(input_data_path.value.values())[0]
        input_content = input_file_info['content']
        
        try:
            input_df = pd.read_csv(pd.io.common.BytesIO(input_content))
        except Exception as e:
            print(f"Error reading CSV file: {e}")
            return
        
        model = model_data['model']
        
        if task == 'Regression':
            scaler = model_data['scaler']
            scaled_features = scaler.transform(input_df)
            predictions = model.predict(scaled_features)
            input_df['predictions'] = predictions
        
        elif task == 'Classification':
            feature_scaler = model_data['featureScaler']
            label_scaler = model_data['labelScaler']
            scaled_features = feature_scaler.transform(input_df)
            predictions_encoded = model.predict(scaled_features)
            predictions = label_scaler.inverse_transform(predictions_encoded)
            input_df['predictions'] = predictions

        print("Predictions:")
        display(input_df)

inference_button = widgets.Button(description="Perform Inference")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        perform_inference()

inference_button.on_click(on_button_clicked)

display(inference_button, output)

Button(description='Perform Inference', style=ButtonStyle())

Output()